In [1]:
from depolarizing_plugin import DepolarizingPlugin


from qat.lang.AQASM import H, X, RX, Program, CNOT
from qat.qpus import get_default_qpu

prog = Program()
reg = prog.qalloc(2)
prog.apply(H, reg[0])
prog.apply(X, reg[1])
prog.apply(CNOT, reg)
prog.apply(RX(0.4), reg[0])

circ = prog.to_circ()

prob_1q = 0.0
prob_2q = 0.05

qpu = get_default_qpu()
depol = DepolarizingPlugin(prob_1qb=prob_1q, prob_2qb=prob_2q, n_samples=5000)
res = (depol|qpu).submit(circ.to_job())
for sample in res:
    print(sample.state, sample.probability)

print("====")
res = qpu.submit(circ.to_job())
for sample in res:
    print(sample.state, sample.probability)

15
norm final distrib= 1.0000000000000222
|00> 0.03143222612311677
|01> 0.46856777387689436
|10> 0.46856777387689436
|11> 0.03143222612311677
====
|00> 0.019734751499278724
|01> 0.4802652485007212
|10> 0.4802652485007212
|11> 0.019734751499278724


## Same computation, with qat.noisy simulator

In [2]:
from qat.qpus import NoisyQProc
from qat.quops import make_depolarizing_channel

depol1 = make_depolarizing_channel(prob_1q, 1, 'equal_probs')
depol2 = make_depolarizing_channel(prob_2q, 2, 'equal_probs')

from qat.hardware import DefaultHardwareModel
gate_noise_dict = {}
for gname in ["X", "H"]:
    gate_noise_dict[gname] = lambda: depol1 
for gname in ["RX"]:
    gate_noise_dict[gname] = lambda _: depol1 
for gname in ["CNOT"]:
    gate_noise_dict[gname] = lambda: depol2
hw_model = DefaultHardwareModel(gate_noise=gate_noise_dict)

noisy_qpu = NoisyQProc(hardware_model=hw_model, 
                       #sim_method="stochastic"
                      )

print("====")
res = noisy_qpu.submit(circ.to_job())
for sample in res:
    print(sample.state, sample.probability)


====
|00> 0.03201556475265052
|01> 0.4679844352473491
|10> 0.4679844352473492
|11> 0.03201556475265052
